In [ ]:
import matplotlib.pyplot as pl
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm

import fair21
from fair21.interface import fill, initialise

In [ ]:
TIME_AXIS=0
SCENARIO_AXIS=1
CONFIG_AXIS=2
SPECIES_AXIS=3
GASBOX_AXIS=4

IIRF_MAX=100

# this could be converted straight to xarray
earth_radius = 6371000 # m
mass_atmosphere = 5.1352e18 # mass of atmosphere, kg
seconds_per_year = 60 * 60 * 24 * 365.24219

In [ ]:
# defined at top level

species_types = [
    'co2 ffi',
    'co2 afolu',
    'co2',
    'ch4',
    'n2o',
    'cfc-11',
    'other halogen',
    'f-gas',
    'sulfur',
    'black carbon',
    'organic carbon',
    'other slcf',
    'nox aviation',
    'ozone',
    'aerosol-radiation interactions',
    'aerosol-cloud interactions',
    'contrails',
    'lapsi',
    'stratospheric water vapour',
    'land use'
    'volcanic',
    'solar',
    'unspecified'
]

valid_input_modes = {
    'co2 ffi': ['emissions'],
    'co2 afolu': ['emissions'],
    'co2': ['calculated', 'concentration', 'forcing'],
    'ch4': ['emissions', 'concentration', 'forcing'],
    'n2o': ['emissions', 'concentration', 'forcing'],
    'cfc-11': ['emissions', 'concentration', 'forcing'],
    'other halogen': ['emissions', 'concentration', 'forcing'],
    'f-gas': ['emissions', 'concentration', 'forcing'],
    'sulfur': ['emissions'],
    'black carbon': ['emissions'],
    'organic carbon': ['emissions'],
    'other slcf': ['emissions'],
    'nox aviation': ['emissions'],
    'ozone': ['calculated', 'forcing'],
    'aerosol-radiation interactions': ['calculated', 'forcing'],
    'aerosol-cloud interactions': ['calculated', 'forcing'],
    'contrails': ['calculated', 'forcing'],
    'lapsi': ['calculated', 'forcing'],
    'stratospheric water vapour': ['calculated', 'forcing'],
    'land use': ['calculated', 'forcing'],
    'volcanic': ['forcing'],
    'solar': ['forcing'],
    'unspecified': ['forcing'],
}

multiple_allowed = {
    'co2 ffi': False,
    'co2 afolu': False,
    'co2': False,
    'ch4': False,
    'n2o': False,
    'cfc-11': False,
    'other halogen': True,
    'f-gas': True,
    'sulfur': False,
    'black carbon': False,
    'organic carbon': False,
    'other slcf': True,
    'nox aviation': False,
    'ozone': False,
    'aerosol-radiation interactions': False,
    'aerosol-cloud interactions': False,
    'contrails': False,
    'lapsi': False,
    'stratospheric water vapour': False,
    'land use': False,
    'volcanic': False,
    'solar': False,
    'unspecified': True,
}

In [ ]:
class FAIR:
    
    def __init__(self):
        pass
    
    def define_time(self, start, end, step):
        self.timebounds = np.arange(start, end+step, step)
        self.timepoints = 0.5 * (self.timebounds[1:] + self.timebounds[:-1])
        self.timestep = step
        self._n_timebounds = len(self.timebounds)
        self._n_timepoints = len(self.timepoints)
        
    def define_scenarios(self, scenarios):
        self.scenarios = scenarios
        self._n_scenarios = len(scenarios)
        
    def define_configs(self, configs):
        self.configs = configs
        self._n_configs = len(configs)
    
    def define_species(self, species, properties):
        self.species = species
        self._n_species = len(species)
        self.properties = properties
                
        # 1. everything we want to run with defined?
        for specie in species:
            if specie not in properties:
                raise ValueError(f"{specie.name} does not have a corresponding key in `properties`.")

            # 2. everything a valid species type?
            if properties[specie]['type'] not in species_types:
                raise ValueError(f"{properties[specie]['type']} is not a valid species type. Valid types are: {[t for t in species_types]}")

            # 3. input_modes valid?
            if properties[specie]['input_mode'] not in valid_input_modes[properties[specie]['type']]:
                raise ValueError(f"{properties[specie]['input_mode']} is not a valid input mode for {properties[specie]['type']}. Valid input modes are: {[m for m in valid_input_modes[properties[specie]['type']]]}")

    
    def run_control(
        self,
        n_gasboxes=4,
        n_layers=3, 
        aci_method='smith2021',
        ghg_method='meinshausen2020',
        ch4_method='leach2021'
    ):
        self.gasboxes = range(n_gasboxes)
        self.layers = range(n_layers)
        aci_method=aci_method.lower()
        if aci_method in ['smith2021', 'stevens2015']:
            self.aci_method = aci_method
        else:
            raise ValueError(f"aci_method should be smith2021 or stevens2015; you provided {aci_method}.")
        if ghg_method in ['leach2021', 'meinshausen2020', 'etminan2016', 'myhre1998']:
            self.ghg_method = ghg_method
        else:
            raise ValueError(f"`ghg_method` should be one of [leach2021, meinshausen2020, etminan2016, myhre1998]; you provided {ghg_method}.")
        self._n_gasboxes = n_gasboxes
        self._n_layers = n_layers
        #self._n_aci_parameters = 3 if aci_method=='smith2021' else 2
        self.aci_parameters = ['scale', 'Sulfur', 'BC+OC']#[:self._n_aci_parameters]
        
        
    def allocate(self):
        
        # driver/output variables
        self.emissions = xr.DataArray(
            np.ones((self._n_timepoints, self._n_scenarios, self._n_configs, self._n_species)) * np.nan,
            coords = (self.timepoints, self.scenarios, self.configs, self.species),
            dims = ('timepoints', 'scenario', 'config', 'specie')
        )
        self.concentration = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs, self._n_species)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs, self.species),
            dims = ('timebounds', 'scenario', 'config', 'specie')
        )
        self.forcing = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs, self._n_species)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs, self.species),
            dims = ('timebounds', 'scenario', 'config', 'specie')
        )
        self.temperature = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs, self._n_layers)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs, self.layers),
            dims = ('timebounds', 'scenario', 'config', 'layer')
        )
        
        # output variables
        self.airborne_emissions = xr.DataArray(
            np.zeros((self._n_timebounds, self._n_scenarios, self._n_configs, self._n_species)),
            coords = (self.timebounds, self.scenarios, self.configs, self.species),
            dims = ('timebounds', 'scenario', 'config', 'specie')
        )

        self.alpha_lifetime = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs, self._n_species)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs, self.species),
            dims = ('timebounds', 'scenario', 'config', 'specie')
        )
        
        self.cumulative_emissions = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs, self._n_species)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs, self.species),
            dims = ('timebounds', 'scenario', 'config', 'specie')
        )

        self.airborne_fraction = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs, self._n_species)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs, self.species),
            dims = ('timebounds', 'scenario', 'config', 'specie')
        )
        
        self.ocean_heat_content_change = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs),
            dims = ('timebounds', 'scenario', 'config')
        )

        self.toa_imbalance = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs),
            dims = ('timebounds', 'scenario', 'config')
        )
        
        self.ocean_heat_content_change = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs),
            dims = ('timebounds', 'scenario', 'config')
        )

        self.stochastic_forcing = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs),
            dims = ('timebounds', 'scenario', 'config')
        )
        
        self.forcing_sum = xr.DataArray(
            np.ones((self._n_timebounds, self._n_scenarios, self._n_configs)) * np.nan,
            coords = (self.timebounds, self.scenarios, self.configs),
            dims = ('timebounds', 'scenario', 'config')
        )
        
        # climate configs
        self.climate_configs = xr.Dataset(
            {
                'ocean_heat_transfer': (["config", "layer"], np.ones((self._n_configs, self._n_layers)) * np.nan),
                'ocean_heat_capacity': (["config", "layer"], np.ones((self._n_configs, self._n_layers)) * np.nan),
                'deep_ocean_efficacy': ("config", np.ones(self._n_configs) * np.nan),
                'stochastic_run': ("config", np.zeros(self._n_configs, dtype=bool)),
                'sigma_eta': ("config", np.ones(self._n_configs) * 0.5),
                'sigma_xi': ("config", np.ones(self._n_configs) * 0.5),
                'gamma_autocorrelation': ("config", np.ones(self._n_configs) * 2),
                'seed': ("config", np.zeros(self._n_configs, dtype=np.uint32)),
                'use_seed': ("config", np.zeros(self._n_configs, dtype=bool)),
                'forcing_4co2': ("config", np.ones(self._n_configs) * 8),
            },
            coords = {
                "config": self.configs,
                "layer": self.layers
            },
        )
        
        # species configs
        self.species_configs = xr.Dataset(
            {
                # general parameters applicable to all species
                'tropospheric_adjustment': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'forcing_efficacy': (["config", "specie"], np.ones((self._n_configs, self._n_species))),
                'forcing_temperature_feedback': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'forcing_scale': (["config", "specie"], np.ones((self._n_configs, self._n_species))),

                # greenhouse gas parameters
                'partition_fraction': (
                    ["config", "specie", "gasbox"], np.ones((self._n_configs, self._n_species, self._n_gasboxes)) * np.nan
                ),
                'unperturbed_lifetime': (
                    ["config", "specie", "gasbox"], np.ones((self._n_configs, self._n_species, self._n_gasboxes)) * np.nan
                ),
                'molecular_weight': ("specie", np.ones(self._n_species) * np.nan),
                'baseline_concentration': (["config", "specie"], np.ones((self._n_configs, self._n_species)) * np.nan),
                'iirf_0': (["config", "specie"], np.ones((self._n_configs, self._n_species)) * np.nan),
                'iirf_airborne': (["config", "specie"], np.ones((self._n_configs, self._n_species)) * np.nan),
                'iirf_uptake': (["config", "specie"], np.ones((self._n_configs, self._n_species)) * np.nan),
                'iirf_temperature': (["config", "specie"], np.ones((self._n_configs, self._n_species)) * np.nan),
                'baseline_emissions': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'g0': (["config", "specie"], np.ones((self._n_configs, self._n_species)) * np.nan),
                'g1': (["config", "specie"], np.ones((self._n_configs, self._n_species)) * np.nan),
                
                # general parameters relating emissions, concentration or forcing of one species to forcing of another
                # these are all linear factors
                'greenhouse_gas_radiative_efficiency': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'contrails_radiative_efficiency': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'erfari_radiative_efficiency': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'h2o_stratospheric_factor': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'lapsi_radiative_efficiency': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'land_use_cumulative_emissions_to_forcing': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),
                'ozone_radiative_efficiency': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),

                # specific parameters for ozone-depleting GHGs
                'cl_atoms': ("specie", np.zeros(self._n_species)),
                'br_atoms': ("specie", np.zeros(self._n_species)),
                'fractional_release': (["config", "specie"], np.zeros((self._n_configs, self._n_species))),

                # specific parameters for aerosol-cloud interactions
                'aci_parameters': (["config", "aci_parameter"], np.ones((self._n_configs, 3)) * np.nan)

            },
            coords = {
                "config": self.configs,
                "specie": self.species,
                "gasbox": self.gasboxes,
                "aci_parameter": self.aci_parameters
            },
        )
        
    # greenhouse gas convenience functions
    def calculate_iirf0(self, iirf_horizon=100):
        gasbox_axis = self.species_configs["partition_fraction"].get_axis_num('gasbox')
        self.species_configs["iirf_0"] = (
            np.sum(self.species_configs["unperturbed_lifetime"] *
            (1 - np.exp(-iirf_horizon / self.species_configs["unperturbed_lifetime"]))
            * self.species_configs["partition_fraction"], gasbox_axis)
        )
        
    def calculate_g(self, iirf_horizon=100):
        gasbox_axis = self.species_configs["partition_fraction"].get_axis_num('gasbox')
        self.species_configs["g1"] = np.sum(
            self.species_configs["partition_fraction"] * self.species_configs["unperturbed_lifetime"] *
            (1 - (1 + iirf_horizon/self.species_configs["unperturbed_lifetime"]) *
            np.exp(-iirf_horizon/self.species_configs["unperturbed_lifetime"])),
        axis=gasbox_axis)
        self.species_configs["g0"] = np.exp(-1 * np.sum((self.species_configs["partition_fraction"])*
            self.species_configs["unperturbed_lifetime"]*
            (1 - np.exp(-iirf_horizon/self.species_configs["unperturbed_lifetime"])), axis=gasbox_axis)/
            self.species_configs["g1"]
        )

    def calculate_concentration_per_emission(self, mass_atmosphere=5.1352e18, molecular_weight_air=28.97):
        self.species_configs["concentration_per_emission"] = 1 / (
            mass_atmosphere / 1e18 * 
            self.species_configs["molecular_weight"] / molecular_weight_air
        )

    def _fill_co2_total_emissions(self):
        # co2 = co2 ffi + co2 afolu
        pass
    
    # climate response
    def make_ebms(self):
        self.ebms = fair21.multi_ebm(
            self.configs,
            ocean_heat_capacity=self.climate_configs['ocean_heat_capacity'],
            ocean_heat_transfer=self.climate_configs['ocean_heat_transfer'],
            deep_ocean_efficacy=self.climate_configs['deep_ocean_efficacy'],
            stochastic_run=self.climate_configs['stochastic_run'],
            sigma_eta=self.climate_configs['sigma_eta'],
            sigma_xi=self.climate_configs['sigma_xi'],
            gamma_autocorrelation=self.climate_configs['gamma_autocorrelation'],
            seed=self.climate_configs['seed'],
            use_seed=self.climate_configs['use_seed'],
            forcing_4co2=self.climate_configs['forcing_4co2'],
            timestep=self.timestep,
            timebounds=self.timebounds,
        )
        
    def _check_properties(self):
        def _raise_if_nan(specie, input_mode):
            raise ValueError(f"{specie} contains NaN values in its {input_mode} array, which you are trying to drive the simulation with.")  

        # check if emissions, concentration, forcing have been defined
            
        for specie in self.species:
            # 4. do we have non-nan data in every case?
            if self.properties[specie]['input_mode'] == 'emissions':
                n_nan = np.isnan(self.emissions.loc[dict(specie=specie)]).sum()
                if n_nan > 0: _raise_if_nan(specie, 'emissions') 
            elif self.properties[specie]['input_mode'] == 'concentration':
                n_nan = np.isnan(self.concentration.loc[dict(specie=specie)]).sum()
                if n_nan > 0: _raise_if_nan(specie, 'concentration')
            elif self.properties[specie]['input_mode'] == 'forcing':
                n_nan = np.isnan(self.forcing.loc[dict(specie=specie)]).sum()
                if n_nan > 0: _raise_if_nan(specie, 'forcing')

        properties_df = pd.DataFrame(properties).T

        # 5. special dependency cases
        if 'co2' in list(properties_df.loc[properties_df['input_mode']=='calculated']['type']):
            if (
                'co2 ffi' not in list(properties_df.loc[properties_df['input_mode']=='emissions']['type']) or
                'co2 afolu' not in list(properties_df.loc[properties_df['input_mode']=='emissions']['type'])
            ):
                raise ValueError('`co2` in `calculated` mode requires `co2 ffi` and `co2 afolu` in `emissions` mode.')

        if 'land use' in list(properties_df.loc[properties_df['input_mode']=='calculated']['type']):
            if 'co2 afolu' not in list(properties_df.loc[properties_df['input_mode']=='emissions']['type']):
                raise ValueError('`land use` in `calculated` mode requires `co2 afolu` in `emissions` mode.')

        if 'aerosol-cloud interactions' in list(properties_df.loc[properties_df['input_mode']=='calculated']['type']):
            if 'sulfur' not in list(properties_df.loc[properties_df['input_mode']=='emissions']['type']):
                raise ValueError('`aerosol-cloud interactions` in `calculated` mode requires `sulfur` in `emissions` mode for `aci_method = stevens2015`.')
            elif (
                self.aci_method=='smith2021' and
                'black carbon' not in list(properties_df.loc[properties_df['input_mode']=='emissions']['type']) and
                'organic carbon' not in list(properties_df.loc[properties_df['input_mode']=='emissions']['type'])
            ):
                raise ValueError('`aerosol-cloud interactions` in `calculated` mode requires `sulfur`, `black carbon` and `organic carbon` in `emissions` mode for `aci_method = smith2021`.')

        co2_to_forcing = False
        ch4_to_forcing = False
        n2o_to_forcing = False

        if 'co2' in list(properties_df.loc[properties_df['input_mode']=='calculated']['type']) or 'co2' in list(properties_df.loc[properties_df['input_mode']=='emissions']['type']) or 'co2' in list(properties_df.loc[properties_df['input_mode']=='concentration']['type']):
            co2_to_forcing=True
        if 'ch4' in list(properties_df.loc[properties_df['input_mode']=='emissions']['type']) or 'ch4' in list(properties_df.loc[properties_df['input_mode']=='concentration']['type']):
            ch4_to_forcing=True
        if 'n2o' in list(properties_df.loc[properties_df['input_mode']=='emissions']['type']) or 'n2o' in list(properties_df.loc[properties_df['input_mode']=='concentration']['type']):
            n2o_to_forcing=True
        if self.ghg_method in ['meinshausen2020', 'etminan2016']:
            if 0 < co2_to_forcing+ch4_to_forcing+n2o_to_forcing < 3:
                raise ValueError("For `ghg_method` either `meinshausen2016` or `etminan2016`, either all of `co2`, `ch4` and `n2o` must be provided in a form that can be converted to concentrations, or none")
        elif self.ghg_method=='myhre1998':
            if 0 < ch4_to_forcing+n2o_to_forcing < 2:
                raise ValueError("For `ghg_method` either `myhre1998`, either both of `ch4` and `n2o` must be provided in a form that can be converted to concentrations, or neither")

        # 6. uniques
        for specie_type in properties_df['type'].unique():
            n_repeats = sum(properties_df['type']==specie_type)
            if n_repeats > 1 and not multiple_allowed[specie_type]:
                raise ValueError(f'{specie_type} is defined {n_repeats} times in the problem, but must be unique.')

        self.properties_df = properties_df
    
    def _make_indices(self):
        # the following are all n_species-length boolean arrays

        # these define what we utimately want input or output from
        self._emissions_species = list(self.properties_df.loc[self.properties_df.loc[:,'emissions']==True].index)
        self._concentration_species = list(self.properties_df.loc[self.properties_df.loc[:,'concentration']==True].index)
        self._forcing_species = list(self.properties_df.loc[self.properties_df.loc[:,'forcing']==True].index)

        # these define which species do what in FaIR
        self._ghg_indices = np.asarray(self.properties_df.loc[:, 'greenhouse_gas'].values, dtype=bool)
        self._ari_precursor_indices = np.asarray(self.properties_df.loc[:, 'aerosol_radiation_precursor'].values, dtype=bool)
        self._aci_precursor_indices = np.asarray(self.properties_df.loc[:, 'aerosol_cloud_precursor'].values, dtype=bool)
        self._co2_ffi_indices = np.asarray(self.properties_df['type']=='co2 ffi', dtype=bool)
        self._co2_afolu_indices = np.asarray(self.properties_df['type']=='co2 afolu', dtype=bool)
        self._co2_indices = np.asarray(self.properties_df['type']=='co2', dtype=bool)
        self._ch4_indices = np.asarray(self.properties_df['type']=='ch4', dtype=bool)
        self._n2o_indices = np.asarray(self.properties_df['type']=='n2o', dtype=bool)
        self._sulfur_indices = np.asarray(self.properties_df['type']=='sulfur', dtype=bool)
        self._bc_indices = np.asarray(self.properties_df['type']=='black carbon', dtype=bool)
        self._oc_indices = np.asarray(self.properties_df['type']=='organic carbon', dtype=bool)
        self._ari_indices = np.asarray(self.properties_df['type']=='aerosol-radiation interactions', dtype=bool)
        self._aci_indices = np.asarray(self.properties_df['type']=='aerosol-cloud interactions', dtype=bool)
        self._minor_ghg_indices = self._ghg_indices ^ self._co2_indices ^ self._ch4_indices ^ self._n2o_indices

        # and these ones are more specific, tripping certain behaviours or functions
        self._ghg_forward_indices = np.asarray(
            (
                (
                    (self.properties_df.loc[:,'input_mode']=='emissions')|
                    (self.properties_df.loc[:,'input_mode']=='calculated')
                ) &
                (self.properties_df.loc[:,'greenhouse_gas'])
            ).values, dtype=bool
        )
        self._ghg_inverse_indices = np.asarray(
            (
                (self.properties_df.loc[:,'input_mode']=='concentration')&
                (self.properties_df.loc[:,'greenhouse_gas'])
            ).values, dtype=bool
        )
        self._ari_from_emissions_indices = np.asarray(
            (
                ~(self.properties_df.loc[:,'greenhouse_gas'])&
                (self.properties_df.loc[:,'aerosol_radiation_precursor'])
            ).values, dtype=bool
        )
        self._ari_from_concentration_indices = np.asarray(
            (
                (self.properties_df.loc[:,'greenhouse_gas'])&
                (self.properties_df.loc[:,'aerosol_radiation_precursor'])
            ).values, dtype=bool
        )
                
    def run(self):
        self._check_properties()
        self._make_indices()
        
        # part of pre-run: TODO move to a new method
        if self._co2_indices.sum() + self._co2_ffi_indices.sum() + self._co2_afolu_indices.sum()==3:
            self.emissions[...,self._co2_indices] = self.emissions[...,self._co2_ffi_indices].data + self.emissions[...,self._co2_afolu_indices].data
        self.cumulative_emissions[1:,...] = self.emissions.cumsum(axis=0, skipna=False) * self.timestep + self.cumulative_emissions[0,...]
        
        # create numpy arrays
        alpha_lifetime_array = self.alpha_lifetime.data
        airborne_emissions_array = self.airborne_emissions.data
        baseline_concentration_array = self.species_configs['baseline_concentration'].data
        baseline_emissions_array = self.species_configs['baseline_emissions'].data
        concentration_array = self.concentration.data
        concentration_per_emission_array = self.species_configs['concentration_per_emission'].data
        cummins_state_array = np.ones((self._n_timebounds, self._n_scenarios, self._n_configs, self._n_layers+1)) * np.nan
        cumulative_emissions_array = self.cumulative_emissions.data
        deep_ocean_efficacy_array = self.climate_configs['deep_ocean_efficacy'].data
        eb_matrix_d_array = self.ebms['eb_matrix_d'].data
        emissions_array = self.emissions.data
        erfari_radiative_efficiency_array = self.species_configs['erfari_radiative_efficiency'].data
        erfaci_scale_array = self.species_configs['aci_parameters'].data[:,0]
        erfaci_shape_sulfur_array = self.species_configs['aci_parameters'].data[:,1]
        erfaci_shape_bcoc_array = self.species_configs['aci_parameters'].data[:,2]
        forcing_array = self.forcing.data
        forcing_scale_array = self.species_configs['forcing_scale'].data
        forcing_efficacy_array = self.species_configs['forcing_efficacy'].data
        forcing_efficacy_sum_array = np.ones((self._n_timebounds, self._n_scenarios, self._n_configs)) * np.nan
        forcing_sum_array = self.forcing_sum.data
        forcing_vector_d_array = self.ebms['forcing_vector_d'].data
        g0_array = self.species_configs['g0'].data
        g1_array = self.species_configs['g1'].data
        gas_partitions_array = np.zeros((self._n_scenarios, self._n_configs, self._n_species, self._n_gasboxes))
        greenhouse_gas_radiative_efficiency_array = self.species_configs['greenhouse_gas_radiative_efficiency'].data
        iirf_0_array = self.species_configs['iirf_0'].data
        iirf_airborne_array = self.species_configs['iirf_airborne'].data
        iirf_temperature_array = self.species_configs['iirf_temperature'].data
        iirf_uptake_array = self.species_configs['iirf_uptake'].data
        iirf_temperature = self.species_configs['iirf_temperature'].data
        ocean_heat_transfer_array = self.climate_configs['ocean_heat_transfer'].data
        partition_fraction_array = self.species_configs['partition_fraction'].data
        stochastic_d_array = self.ebms['stochastic_d'].data
        unperturbed_lifetime_array = self.species_configs['unperturbed_lifetime'].data
        
        # forcing should be initialised so this should not be nan. We could check, or allow silent fail as some species don't take forcings and would correctly be nan.
        forcing_sum_array[0:1, ...] = np.nansum(
            forcing_array[0:1, ...], axis=SPECIES_AXIS
        )
        
        # this is the most important state vector
        cummins_state_array[0, ..., 0] = forcing_sum_array[0, ...]
        cummins_state_array[..., 1:] = self.temperature.data
        
        # it's all been leading to this : FaIR MAIN LOOP
        for i_timepoint in tqdm(range(self._n_timepoints)):     
    
            # 1. alpha scaling
            alpha_lifetime_array[i_timepoint:i_timepoint+1, ..., self._ghg_indices] = calculate_alpha(   # this timepoint
                airborne_emissions_array[i_timepoint:i_timepoint+1, ..., self._ghg_indices],  # last timebound
                cumulative_emissions_array[i_timepoint:i_timepoint+1, ..., self._ghg_indices],  # last timebound
                g0_array[None, None, ..., self._ghg_indices],
                g1_array[None, None, ..., self._ghg_indices],
                iirf_0_array[None, None, ..., self._ghg_indices],
                iirf_airborne_array[None, None, ..., self._ghg_indices],
                iirf_temperature_array[None, None, ..., self._ghg_indices],
                iirf_uptake_array[None, None, ..., self._ghg_indices],
                cummins_state_array[i_timepoint:i_timepoint+1, ..., 1:2],
                IIRF_MAX
            )
            
            # 2. methane lifetime here
            # 3. emissions to concentrations
            (
                concentration_array[i_timepoint+1:i_timepoint+2, ..., self._ghg_forward_indices], 
                gas_partitions_array[..., self._ghg_forward_indices, :], 
                airborne_emissions_array[i_timepoint+1:i_timepoint+2, ..., self._ghg_forward_indices]
            ) = step_concentration( 
                emissions_array[i_timepoint:i_timepoint+1, ..., self._ghg_forward_indices, None],  # this timepoint
                gas_partitions_array[..., self._ghg_forward_indices, :], # last timebound
                airborne_emissions_array[i_timepoint+1:i_timepoint+2, ..., self._ghg_forward_indices, None],  # last timebound
                alpha_lifetime_array[i_timepoint:i_timepoint+1, ..., self._ghg_forward_indices, None],
                baseline_concentration_array[None, None, ..., self._ghg_forward_indices],
                baseline_emissions_array[None, None, ..., self._ghg_forward_indices, None],
                concentration_per_emission_array[None, None, ..., self._ghg_forward_indices],
                unperturbed_lifetime_array[None, None, ..., self._ghg_forward_indices, :],
        #        oxidation_matrix,
                partition_fraction_array[None, None, ..., self._ghg_forward_indices, :],
                self.timestep,
            )
            # 4. concentrations to emissions
            (
                emissions_array[i_timepoint:i_timepoint+1, ..., self._ghg_inverse_indices], 
                gas_partitions_array[..., self._ghg_inverse_indices, :], 
                airborne_emissions_array[i_timepoint+1:i_timepoint+2, ..., self._ghg_inverse_indices]
            ) = unstep_concentration( 
                concentration_array[i_timepoint+1:i_timepoint+2, ..., self._ghg_inverse_indices],  # this timepoint
                gas_partitions_array[None, ..., self._ghg_inverse_indices, :], # last timebound
                airborne_emissions_array[i_timepoint:i_timepoint+1, ..., self._ghg_inverse_indices, None],  # last timebound
                alpha_lifetime_array[i_timepoint:i_timepoint+1, ..., self._ghg_inverse_indices, None],
                baseline_concentration_array[None, None, ..., self._ghg_inverse_indices],
                baseline_emissions_array[None, None, ..., self._ghg_inverse_indices],
                concentration_per_emission_array[None, None, ..., self._ghg_inverse_indices],
                unperturbed_lifetime_array[None, None, ..., self._ghg_inverse_indices, :],
        #        oxidation_matrix,
                partition_fraction_array[None, None, ..., self._ghg_inverse_indices, :],
                self.timestep,
            )
            cumulative_emissions_array[i_timepoint+1, ..., self._ghg_inverse_indices] = cumulative_emissions_array[i_timepoint, ..., self._ghg_inverse_indices] + emissions_array[i_timepoint, ..., self._ghg_inverse_indices] * self.timestep

            # 5. greenhouse gas concentrations to forcing
            forcing_array[i_timepoint+1:i_timepoint+2, ..., self._ghg_indices] = meinshausen(
                concentration_array[i_timepoint+1:i_timepoint+2, ...],
                baseline_concentration_array[None, None, ...] * np.ones((1, self._n_scenarios, self._n_configs, self._n_species)),
                forcing_scale_array[None, None, ...],
                greenhouse_gas_radiative_efficiency_array[None, None, ...],
                self._co2_indices,
                self._ch4_indices,
                self._n2o_indices,
                self._minor_ghg_indices,
            )[0:1, ..., self._ghg_indices]

            # 6. aerosol direct forcing
            forcing_array[i_timepoint+1:i_timepoint+2, ..., self._ari_indices] = calculate_erfari_forcing(
                emissions_array[i_timepoint:i_timepoint+1, ...],
                concentration_array[i_timepoint+1:i_timepoint+2, ...],
                baseline_emissions_array[None, None, ...],
                baseline_concentration_array[None, None, ...],
                forcing_scale_array[None, None, ...],
                erfari_radiative_efficiency_array[None, None, ...],
                self._ari_from_emissions_indices,
                self._ari_from_concentration_indices,
            )

            # 7. aerosol indirect forcing
            forcing_array[i_timepoint+1:i_timepoint+2, ..., self._aci_indices] = calculate_erfaci_forcing(
                emissions_array[i_timepoint:i_timepoint+1, ...],
                baseline_emissions_array[None, None, ...],
                forcing_scale_array[None, None, ..., self._aci_indices],
                erfaci_scale_array[None, None, :, None],
                erfaci_shape_sulfur_array[None, None, :, None],
                erfaci_shape_bcoc_array[None, None, :, None],
                self._sulfur_indices,
                self._bc_indices,
                self._oc_indices,
                self.aci_method
            )

            # 8. ozone
            # 9. contrails from NOx
            # 10. BC and OC to LAPSI
            # 11. CH4 to stratospheric water vapour
            # 12. CO2 cumulative to land use change 
            # 13. volcanic forcing temperature dependence?
            # 14. sum forcings
            forcing_sum_array[i_timepoint+1:i_timepoint+2, ...] = np.nansum(
                forcing_array[i_timepoint+1:i_timepoint+2, ...], axis=SPECIES_AXIS
            )
            forcing_efficacy_sum_array[i_timepoint+1:i_timepoint+2, ...]=np.nansum(
                forcing_array[i_timepoint+1:i_timepoint+2, ...]*forcing_efficacy_array[None, None, ...], axis=SPECIES_AXIS
            )

            # 15. forcing to temperature
            #if not self.run_config.temperature_prescribed:
            cummins_state_array[i_timepoint+1:i_timepoint+2, ...] = step_temperature(
                cummins_state_array[i_timepoint:i_timepoint+1, ...],
                eb_matrix_d_array[None, None, ...],
                forcing_vector_d_array[None, None, ...],
                stochastic_d_array[i_timepoint+1:i_timepoint+2, None, ...],
                forcing_efficacy_sum_array[i_timepoint+1:i_timepoint+2, ..., None]
            )

        # 16. TOA imbalance
        toa_imbalance_array = calculate_toa_imbalance_postrun(
            cummins_state_array,
            forcing_sum_array,#[..., None],  # not efficacy adjusted, is this correct?
            ocean_heat_transfer_array,
            deep_ocean_efficacy_array,
        )

        # 17. Ocean heat content change
        ocean_heat_content_change_array = (
            np.cumsum(toa_imbalance_array * self.timestep, axis=TIME_AXIS) * earth_radius**2 * 4 * np.pi * seconds_per_year
        )

        # 18. calculate airborne fraction - we have NaNs and zeros we know about, and we don't mind
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            airborne_fraction_array = airborne_emissions_array / cumulative_emissions_array

        # 19. (Re)allocate to xarray
        self.temperature.data = cummins_state_array[..., 1:]
        self.concentration.data = concentration_array
        self.emissions.data = emissions_array
        self.forcing.data = forcing_array
        self.forcing_sum.data = forcing_sum_array
        self.cumulative_emissions.data = cumulative_emissions_array
        self.airborne_emissions.data = airborne_emissions_array
        self.airborne_fraction.data = airborne_fraction_array
        self.ocean_heat_content_change.data = ocean_heat_content_change_array
        self.toa_imbalance.data = toa_imbalance_array
        self.stochastic_forcing.data = cummins_state_array[..., 0]

In [ ]:
properties = {
    'CO2 FFI': {
        'type': 'co2 ffi',
        'emissions': True,
        'concentration': False,
        'forcing': False,
        'input_mode': 'emissions',
        'greenhouse_gas': False,  # it doesn't behave as a GHG in the model
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    },
    'CO2 AFOLU': {
        'type': 'co2 afolu',
        'emissions': True,
        'concentration': False,
        'forcing': False,
        'input_mode': 'emissions',
        'greenhouse_gas': False,  # it doesn't behave as a GHG in the model
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    },
    'CO2': {
        'type': 'co2',
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input_mode': 'calculated',
        'greenhouse_gas': True,
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    },
    'CH4': {
        'type': 'ch4',
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_radiation_precursor': True,
        'aerosol_cloud_precursor': False,
    },
    'N2O': {
        'type': 'n2o',
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_radiation_precursor': True,
        'aerosol_cloud_precursor': False,
    },
    'Sulfur': {
        'type': 'sulfur',
        'emissions': True,
        'concentration': False,
        'forcing': True,
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_radiation_precursor': True,
        'aerosol_cloud_precursor': True,
    },
    'Aerosol-radiation interactions': {
        'type': 'aerosol-radiation interactions',
        'emissions': False,
        'concentration': False,
        'forcing': True,
        'input_mode': 'calculated',
        'greenhouse_gas': False,
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    },
    'Aerosol-cloud interactions': {
        'type': 'aerosol-cloud interactions',
        'emissions': False,
        'concentration': False,
        'forcing': True,
        'input_mode': 'calculated',
        'greenhouse_gas': False,
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    }
}

In [ ]:
species = [
    'CO2 FFI',
    'CO2 AFOLU',
    'CO2',
    'CH4',
    'N2O',
    'Sulfur',
    'Aerosol-radiation interactions',
    'Aerosol-cloud interactions'
]

In [ ]:
f = FAIR()
f.define_time(2000, 2050, 1)
f.define_scenarios(['abrupt', 'ramp'])
f.define_configs(['high', 'central', 'low'])
f.define_species(species, properties)
f.run_control(aci_method='Stevens2015')

In [ ]:
f.allocate()

## Fill in `climate_configs`

These describe how temperature responds to forcing. Here, we will also use the `fill` function to fill in the `configs`. Note you can pass multiple dimensions at once by specifying multiple `kwargs` to `fill`, each corresponding to a dimension of the variable we are filling.

In [ ]:
fill(f.climate_configs["ocean_heat_transfer"], [0.6, 1.3, 1.0], config='high')
fill(f.climate_configs["ocean_heat_capacity"], [5, 15, 80], config='high')
fill(f.climate_configs["deep_ocean_efficacy"], 1.29, config='high')

fill(f.climate_configs["ocean_heat_transfer"], [1.1, 1.6, 0.9], config='central')
fill(f.climate_configs["ocean_heat_capacity"], [8, 14, 100], config='central')
fill(f.climate_configs["deep_ocean_efficacy"], 1.1, config='central')

fill(f.climate_configs["ocean_heat_transfer"], [1.7, 2.0, 1.1], config='low')
fill(f.climate_configs["ocean_heat_capacity"], [6, 11, 75], config='low')
fill(f.climate_configs["deep_ocean_efficacy"], 0.8, config='low')

In [ ]:
#fill(fair.climate_configs["ocean_heat_transfer"], [0.6, 1.3, 1.0], specie='CO2')

In [ ]:
#fill(fair.climate_configs["ocean_heat_transfer"], [1.7, 2.0, 1.1, 15], config='low')

Alternatively we can work directly with `xarray`, though this does not contain validation and error checking.

In [ ]:
# fair.climate_configs["ocean_heat_transfer"].loc[dict(config='high')] = np.array([0.6, 1.3, 1.0])
# fair.climate_configs["ocean_heat_capacity"].loc[dict(config='high')] = np.array([5, 15, 80])
# fair.climate_configs["deep_ocean_efficacy"].loc[dict(config='high')] = 1.29

# fair.climate_configs["ocean_heat_transfer"].loc[dict(config='central')] = np.array([1.1, 1.6, 0.9])
# fair.climate_configs["ocean_heat_capacity"].loc[dict(config='central')] = np.array([8, 14, 100])
# fair.climate_configs["deep_ocean_efficacy"].loc[dict(config='central')] = 1.1

# fair.climate_configs["ocean_heat_transfer"].loc[dict(config='low')] = np.array([1.7, 2.0, 1.1])
# fair.climate_configs["ocean_heat_capacity"].loc[dict(config='low')] = np.array([6, 11, 75])
# fair.climate_configs["deep_ocean_efficacy"].loc[dict(config='low')] = 0.8

In [ ]:
#fair.climate_configs

## Fill in `species_configs`

The basic information we need about a greenhouse gas is its lifetime, partition fraction, baseline concentration and molecular weight.

In [ ]:
fill(f.species_configs["partition_fraction"], [0.2173, 0.2240, 0.2824, 0.2763], specie='CO2')

non_co2_ghgs = ["CH4", "N2O"]   # make a param?
for gas in non_co2_ghgs:
    fill(f.species_configs["partition_fraction"], [1, 0, 0, 0], specie=gas)

fill(f.species_configs["unperturbed_lifetime"], [1e9, 394.4, 36.54, 4.304], specie="CO2")
fill(f.species_configs["unperturbed_lifetime"], 8.25, specie="CH4")
fill(f.species_configs["unperturbed_lifetime"], 109, specie="N2O")
    
fill(f.species_configs["baseline_concentration"], 278.3, specie="CO2")
fill(f.species_configs["baseline_concentration"], 729, specie="CH4")
fill(f.species_configs["baseline_concentration"], 270.3, specie="N2O")

fill(f.species_configs["molecular_weight"], 44.009, specie="CO2")
fill(f.species_configs["molecular_weight"], 16.043, specie="CH4")
fill(f.species_configs["molecular_weight"], 44.013, specie="N2O")

### Greenhouse gas state-dependence

`iirf_0` is the baseline time-integrated airborne fraction (usually over 100 years). It can be calculated from the variables above, but sometimes we might want to change these values.

In [ ]:
f.calculate_iirf0()
f.calculate_g()
f.calculate_concentration_per_emission()

In [ ]:
f.species_configs['iirf_0']

In [ ]:
# override CO2 iirf0 as value as calculation is for present day and we want PI
fill(f.species_configs["iirf_0"], 29, specie='CO2')

In [ ]:
f.species_configs["iirf_airborne"]

In [ ]:
fill(f.species_configs["iirf_airborne"], [0.000819*2, 0.000819, 0], specie='CO2')
fill(f.species_configs["iirf_uptake"], [0.00846*2, 0.00846, 0], specie='CO2')
fill(f.species_configs["iirf_temperature"], [8, 4, 0], specie='CO2')

In [ ]:
fill(f.species_configs['iirf_airborne'], 0.00032, specie='CH4')
fill(f.species_configs['iirf_airborne'], -0.0065, specie='N2O')

In [ ]:
fill(f.species_configs['iirf_uptake'], 0, specie='N2O')
fill(f.species_configs['iirf_uptake'], 0, specie='CH4')

In [ ]:
fill(f.species_configs['iirf_temperature'], -0.3, specie='CH4')
fill(f.species_configs['iirf_temperature'], 0, specie='N2O')

### Aerosol emissions or concentrations to forcing

In [ ]:
fill(f.species_configs["erfari_radiative_efficiency"], -0.0036167830509091486, specie='Sulfur') # W m-2 MtSO2-1 yr
fill(f.species_configs["erfari_radiative_efficiency"], -0.002653/1023.2219696044921, specie='CH4') # W m-2 ppb-1
fill(f.species_configs["erfari_radiative_efficiency"], -0.00209/53.96694437662762, specie='N2O') # W m-2 ppb-1

In [ ]:
fill(f.species_configs["aci_parameters"], 2.09841432, aci_parameter='scale')
fill(f.species_configs["aci_parameters"], 260.34644166, aci_parameter='Sulfur')

## Fill in scenario drivers

for example, `emissions` or `concentrations`.

In [ ]:
fill(f.emissions, 38, scenario='abrupt', specie='CO2 FFI')
fill(f.emissions, 3, scenario='abrupt', specie='CO2 AFOLU')
fill(f.emissions, 100, scenario='abrupt', specie='Sulfur')
fill(f.concentration, 1800, scenario='abrupt', specie='CH4')
fill(f.concentration, 325, scenario='abrupt', specie='N2O')

for config in f.configs:
    fill(f.emissions, np.linspace(0, 38, 50), scenario='ramp', config=config, specie='CO2 FFI')
    fill(f.emissions, np.linspace(0, 3, 50), scenario='ramp', config=config, specie='CO2 AFOLU')
    fill(f.emissions, np.linspace(2.2, 100, 50), scenario='ramp', config=config, specie='Sulfur')
    fill(f.concentration, np.linspace(729, 1800, 51), scenario='ramp', config=config, specie='CH4')
    fill(f.concentration, np.linspace(270, 325, 51), scenario='ramp', config=config, specie='N2O')

In [ ]:
fill(f.species_configs["greenhouse_gas_radiative_efficiency"], 1.3344985680386619e-05, specie='CO2')
fill(f.species_configs["greenhouse_gas_radiative_efficiency"], 0.00038864402860869495, specie='CH4')
fill(f.species_configs["greenhouse_gas_radiative_efficiency"], 0.00319550741640458, specie='N2O')

In [ ]:
f.make_ebms()

In [ ]:
f.ebms

## Initial conditions

In [ ]:
# Define first timestep
initialise(f.concentration, 278.3, specie='CO2')
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
# this should both go in pre_run()
#f.cumulative_emissions[1:,...] = f.emissions.cumsum(axis=0, skipna=False) * f.timestep + f.cumulative_emissions[0,...]

## convert to numpy arrays

Here we transition from our nice clean OO format to a bunch of numpy arrays.

### First some top level error checking and validation

Note that `CO2 FFI` and `CO2 AFOLU` are not taken to be greenhouse gases; emissions of these compounds both go on to form `CO2` which is the greenhouse gas.

#### note to chris: consider raising a specific error

In [ ]:
f.aci_method

### Define the run control array indices

In [ ]:
# this also needs to go pre-run


### Now, we can convert our `species` into arrays

For now do this outside the class and then move to FAIR.run()

In [ ]:
# array function - not xarray

import warnings

def calculate_alpha(
    airborne_emissions,
    cumulative_emissions,
    g0,
    g1,
    iirf_0,
    iirf_airborne,
    iirf_temperature,
    iirf_uptake,
    temperature,
    iirf_max=100,
):

    iirf = iirf_0 + iirf_uptake * (cumulative_emissions-airborne_emissions) + iirf_temperature * temperature + iirf_airborne * airborne_emissions
    iirf = (iirf>iirf_max) * iirf_max + iirf * (iirf<iirf_max)
    # overflow and invalid value errors occur with very large and small values
    # in the exponential. This happens with very long lifetime GHGs. Usually
    # these GHGs don't have a temperature dependence on IIRF but even if they
    # did the lifetimes are so long that it is unlikely to have an effect.
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        alpha = g0 * np.exp(iirf / g1)
#        alpha[np.isnan(alpha)]=1
    return alpha

In [ ]:
# array function

GASBOX_AXIS=4

def step_concentration(
    emissions,
    gasboxes_old,
    airborne_emissions_old,
    alpha_lifetime,
    baseline_concentration,
    baseline_emissions,
    concentration_per_emission,
    lifetime,
    partition_fraction,
    timestep,
):

    decay_rate = timestep/(alpha_lifetime * lifetime)
    decay_factor = np.exp(-decay_rate)

    # additions and removals
    gasboxes_new = (
        partition_fraction *
        (emissions - baseline_emissions) *
        1 / decay_rate *
        (1 - decay_factor) * timestep + gasboxes_old * decay_factor
    )

    airborne_emissions_new = np.sum(gasboxes_new, axis=GASBOX_AXIS)
    concentration_out = baseline_concentration + concentration_per_emission * airborne_emissions_new
    
    return concentration_out, gasboxes_new, airborne_emissions_new

In [ ]:
# array function

def unstep_concentration(
    concentration,
    gasboxes_old,
    airborne_emissions_old,
    alpha_lifetime,
    baseline_concentration,
    baseline_emissions,
    concentration_per_emission,
    lifetime,
    partition_fraction,
    timestep,
):

    decay_rate = timestep/(alpha_lifetime * lifetime)   # [1]
    decay_factor = np.exp(-decay_rate)  # [1]

    airborne_emissions_new = (concentration-baseline_concentration)/concentration_per_emission
    emissions = (
        (airborne_emissions_new - np.sum(gasboxes_old*decay_factor, axis=GASBOX_AXIS)) /
        (np.sum(
            partition_fraction / decay_rate * ( 1. - decay_factor ) * timestep,
            axis=GASBOX_AXIS)
        )
    )
    
    gasboxes_new = timestep * emissions[..., None] * partition_fraction * 1/decay_rate * ( 1. - decay_factor ) + gasboxes_old * decay_factor
    emissions_out = emissions + baseline_emissions

    return emissions_out, gasboxes_new, airborne_emissions_new

In [ ]:
def meinshausen(
    concentration,
    baseline_concentration,
    forcing_scaling,
    radiative_efficiency,
    co2_indices,
    ch4_indices,
    n2o_indices,
    minor_greenhouse_gas_indices,
    a1 = -2.4785e-07,
    b1 = 0.00075906,
    c1 = -0.0021492,
    d1 = 5.2488,
    a2 = -0.00034197,
    b2 = 0.00025455,
    c2 = -0.00024357,
    d2 = 0.12173,
    a3 = -8.9603e-05,
    b3 = -0.00012462,
    d3 = 0.045194,
    ):
    
    erf_out = np.ones_like(concentration) * np.nan

    # easier to deal with smaller arrays
    co2 = concentration[..., co2_indices]
    ch4 = concentration[..., ch4_indices]
    n2o = concentration[..., n2o_indices]
    co2_base = baseline_concentration[..., co2_indices]
    ch4_base = baseline_concentration[..., ch4_indices]
    n2o_base = baseline_concentration[..., n2o_indices]
    
    # CO2
    ca_max = co2_base - b1/(2*a1)
    where_central = np.asarray((co2_base < co2) & (co2 <= ca_max)).nonzero()
    where_low = np.asarray((co2 <= co2_base)).nonzero()
    where_high = np.asarray((co2 > ca_max)).nonzero()
    alpha_p = np.ones_like(co2) * np.nan
    alpha_p[where_central] = d1 + a1*(co2[where_central] - co2_base[where_central])**2 + b1*(co2[where_central] - co2_base[where_central])
    alpha_p[where_low] = d1
    alpha_p[where_high] = d1 - b1**2/(4*a1)
    alpha_n2o = c1*np.sqrt(n2o)
    erf_out[..., co2_indices] = (alpha_p + alpha_n2o) * np.log(co2/co2_base) * (forcing_scaling[..., co2_indices])

    # CH4
    erf_out[..., ch4_indices] = (
        (a3*np.sqrt(ch4) + b3*np.sqrt(n2o) + d3) *
        (np.sqrt(ch4) - np.sqrt(ch4_base))
    )  * (forcing_scaling[..., ch4_indices])
    
    # N2O
    erf_out[..., n2o_indices] = (
        (a2*np.sqrt(co2) + b2*np.sqrt(n2o) + c2*np.sqrt(ch4) + d2) *
        (np.sqrt(n2o) - np.sqrt(n2o_base))
    )  * (forcing_scaling[..., n2o_indices])
    
    # linear for other gases
    # TODO: move to a general linear function
    erf_out[..., minor_greenhouse_gas_indices] = (
        (concentration[..., minor_greenhouse_gas_indices] - baseline_concentration[..., minor_greenhouse_gas_indices])
        * radiative_efficiency[..., minor_greenhouse_gas_indices] * 0.001   # unit handling
    ) * (forcing_scaling[..., minor_greenhouse_gas_indices])
    
    return erf_out

In [ ]:
def calculate_erfari_forcing(
    emissions,
    concentration,
    baseline_emissions,
    baseline_concentration,
    forcing_scaling,
    radiative_efficiency,
    emissions_indices,
    concentration_indices
):

#    # zeros because nansum is slow?
#    erf_out = np.zeros((emissions.shape[0], emissions.shape[1], emissions.shape[2], emissions.shape[3]))
    erf_out = np.ones_like(emissions) * np.nan
    
    # emissions-driven forcers
    erf_out[..., emissions_indices] = (
        (emissions[..., emissions_indices] - baseline_emissions[..., emissions_indices])
        * radiative_efficiency[..., emissions_indices]
    ) * forcing_scaling[..., emissions_indices]

    # concentration-driven forcers
    erf_out[..., concentration_indices] = (
        (concentration[..., concentration_indices] - baseline_emissions[..., concentration_indices])
        * radiative_efficiency[..., concentration_indices]
    ) * forcing_scaling[..., concentration_indices]

    # in future we can retain contributions from each species. Will need one ERFari
    # array index for each species so we don't do this here yet.
    return np.nansum(erf_out, axis=SPECIES_AXIS, keepdims=True)

In [ ]:
def calculate_erfaci_forcing(
    emissions,
    baseline_emissions,
    forcing_scaling,
    scale,
    shape_sulfur,
    shape_bcoc,
    sulfur_index,
    bc_index,
    oc_index,
    aci_method,
):

    sulfur = emissions[..., sulfur_index]
    sulfur_base = baseline_emissions[..., sulfur_index]

    if aci_method=="Smith2021":
        bc = emissions[..., bc_index]
        bc_base = baseline_emissions[..., bc_index]
        oc = emissions[..., oc_index]
        oc_base = baseline_emissions[..., oc_index]

    else:
        bc = bc_base = oc = oc_base = 0
        shape_bcoc = 100  # anything to avoid divide by zero

    # TODO: raise an error if sulfur, BC and OC are not all there
    # TODO: check species going in
    radiative_effect = -scale * np.log(
        1 + sulfur/shape_sulfur +
        (bc + oc)/shape_bcoc
    )
    baseline_radiative_effect = -scale * np.log(
        1 + sulfur_base/shape_sulfur +
        (bc_base + oc_base)/shape_bcoc
    )

    erf_out = (radiative_effect - baseline_radiative_effect) * forcing_scaling
    return erf_out

In [ ]:
def step_temperature(
    state_old,
    eb_matrix_d,
    forcing_vector_d,
    stochastic_d,
    forcing
):
    
    state_new = (
        (eb_matrix_d[0, ...] @ state_old[0, ..., None])[..., 0] +
        forcing_vector_d[0, ...] * forcing[0, ..., 0, None] +
        stochastic_d[0, ...]
    )

    return state_new

In [ ]:
def calculate_toa_imbalance_postrun(
    state,
    forcing,
    ocean_heat_transfer,
    deep_ocean_efficacy,
):
    toa_imbalance = (
        forcing -
        ocean_heat_transfer[..., 0] *
        state[..., 1] +
        (1 - deep_ocean_efficacy) * ocean_heat_transfer[..., -1]
        * (state[..., -2] - state[..., -1])
    )
    return toa_imbalance

In [ ]:
f.run()

In [ ]:
pl.plot(f.timebounds, f.temperature.loc[dict(scenario='ramp', layer=0)])

In [ ]:
pl.plot(f.timebounds, f.concentration.loc[dict(scenario='ramp', specie='CO2')])

In [ ]:
f.concentration